# ESSAI

In [1]:
!crest --version



       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.2, Thu, 29 August 14:20:46, 08/29/2024
       commit (65685a7) compiled by 'usr@d37571da525e'

 crest 3.0.2


In [29]:
import os, sys
import inspect
from pathlib import Path
import tempfile
#from utils import run_command

import rdkit
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
import subprocess as sp
import numpy as np
import torch
import torch.nn as nn

In [30]:
import xtb

In [31]:
smiles='CC12C3C4CC(N13)C2=C4'

In [32]:
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
charge = Chem.rdmolops.GetFormalCharge(mol)
atom_number = mol.GetNumAtoms()
# Write the smile to a file
system = lambda x: run_command(x, verbose=5)

In [33]:

with open('test.smi', 'w') as f:
    f.write(smiles)

In [34]:

system('obabel test.smi --gen3D -O test.xyz')


NameError: name 'run_command' is not defined

In [35]:
OB_process= sp.run('obabel  test.smi -O  pat.xyz --gen3d -h --best --errorlevel 2 --minimize --ff MMFF94s  --steps 15000 --sd --crit 1e-9 --log', shell=True, stdout=sp.PIPE, stderr=sp.PIPE, universal_newlines=True, text=True)
          

In [36]:
OB_process

CompletedProcess(args='obabel  test.smi -O  pat.xyz --gen3d -h --best --errorlevel 2 --minimize --ff MMFF94s  --steps 15000 --sd --crit 1e-9 --log', returncode=0, stdout='', stderr='\nA T O M   T Y P E S\n\nIDX\tTYPE\tRING\n1\t1\tNO\n2\t22\tAL\n3\t22\tAL\n4\t1\tAL\n5\t1\tAL\n6\t20\tAL\n7\t8\tAL\n8\t30\tAL\n9\t2\tAL\n10\t5\tNO\n11\t5\tNO\n12\t5\tNO\n13\t5\tNO\n14\t5\tNO\n15\t5\tNO\n16\t5\tNO\n17\t5\tNO\n18\t5\tNO\n\nF O R M A L   C H A R G E S\n\nIDX\tCHARGE\n1\t0.000000\n2\t0.000000\n3\t0.000000\n4\t0.000000\n5\t0.000000\n6\t0.000000\n7\t0.000000\n8\t0.000000\n9\t0.000000\n10\t0.000000\n11\t0.000000\n12\t0.000000\n13\t0.000000\n14\t0.000000\n15\t0.000000\n16\t0.000000\n17\t0.000000\n18\t0.000000\n\nP A R T I A L   C H A R G E S\n\nIDX\tCHARGE\n1\t0.095000\n2\t0.134000\n3\t-0.037000\n4\t0.233200\n5\t0.000000\n6\t0.347600\n7\t-0.525600\n8\t-0.240000\n9\t-0.257200\n10\t0.000000\n11\t0.000000\n12\t0.000000\n13\t0.100000\n14\t0.000000\n15\t0.000000\n16\t0.000000\n17\t0.000000\n18\t0.150000\

In [37]:
!xtb --version


      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.7.1 (edcfbbe) compiled by 'albert@albert-system' on 2024-07-22

normal termination of xtb


In [38]:
import subprocess as sp
import os

# Vérification du fichier
xyz_file = "pat.xyz"
if not os.path.exists(xyz_file):
    raise FileNotFoundError(f"Le fichier {xyz_file} est introuvable.")

# Commande mise à jour pour xTB 6.7.1
try:
    xtb_process = sp.run(
        ["xtb", xyz_file, "--opt", "vtight", "--gfn", "2", "--alpb", "toluene", "--chrg", "0"],
        stdout=sp.PIPE, stderr=sp.PIPE, text=True, check=True
    )

    # Affichage des sorties pour le debug
    print("xTB Output:", xtb_process.stdout)
    print("xTB Error:", xtb_process.stderr)

except sp.CalledProcessError as e:
    print(f"Erreur lors de l'exécution de xTB: {e}")
    print("Sortie standard:", e.output)
    print("Sortie d'erreur:", e.stderr)


xTB Output:       -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.7.1 (edcfbbe) compiled by 'albert@albert-system' on 2024-07-22

   xtb is free software: you can redistribute it and/or modify it under
   the terms of the GNU Lesser General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.
   
   xtb is distributed in the hope that it will be useful,
   but WITHOUT A

In [10]:

# Run the preliminary xtb: 
command_pre = 'CHARGE={};xtb {} --gfn 2 --opt normal -c $CHARGE --iterations 4000'.format(charge, 'test.xyz')
system(command_pre)

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.3.3 (71d3805) compiled by 'conda@b85dec0bf610' on 2021-01-07

   xtb is free software: you can redistribute it and/or modify it under
   the terms of the GNU Lesser General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.
   
   xtb is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; w

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [13]:

system("rm ./gfnff_charges ./gfnff_topo")

    # Run crest conformer ensemble
command_crest = 'CHARGE={};crest {} -gff -mquick -chrg $CHARGE --noreftopo'.format(charge, 'xtbopt.xyz')
system(command_crest)
system('rm ./gfnff_charges ./gfnff_topo')
system('head -n {} crest_conformers.xyz > crest_best.xyz'.format(atom_number+2))

    # Run the calculation: 
command = 'CHARGE={};xtb {} --opt normal -c $CHARGE --iterations 4000 > out_dump'.format(charge, 'crest_best.xyz')
system(command)

    # Read the output: 
with open('./out_dump', 'r') as f: 
    text_content = f.readlines()


rm: impossible de supprimer './gfnff_charges': Aucun fichier ou dossier de ce type
rm: impossible de supprimer './gfnff_topo': Aucun fichier ou dossier de ce type



       |                                            |
       |                 C R E S T                  |
       |                                            |
       |  Conformer-Rotamer Ensemble Sampling Tool  |
       |          based on the GFN methods          |
       |             P.Pracht, S.Grimme             |
       |          Universitaet Bonn, MCTC           |
       Version 2.12,   Thu 19. Mai 16:32:32 CEST 2022
  Using the xTB program. Compatible with xTB version 6.4.0

   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.

   and for works involving QCG as

   • S.Spicher, C.Plett, P.Pracht, A.Hansen, S.Grimme,
     JCTC, 2022, 18 (5), 3174-3189.

   with help from:
   C.Bannwarth, F.Bohle, S.Ehlert, S.Grimme,
   C.Plett, P.Pracht, S.Spicher

   This program is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   M

rm: impossible de supprimer './gfnff_charges': Aucun fichier ou dossier de ce type
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
